In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import linear_model
from sklearn.metrics import confusion_matrix, classification_report

from rtbm import RTBM, minimizer

import rtbm.layers as layers
import rtbm.model as mdl

import warnings
warnings.filterwarnings('ignore')

from rtbm.costfunctions import mse, crossentropy


Process PoolWorker-6:
Process PoolWorker-3:
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Process PoolWorker-5:
Process PoolWorker-4:
Process PoolWorker-7:
Process PoolWorker-1:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Process PoolWorker-8:
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Traceback (most recent call last):
Process PoolWorker-2:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Traceback (most recent call last):
    self._target(*self._args, **self._kwargs)
    self.run()
    self.run()
    self.run()
  File "/usr/lib/python2.7/multiprocessing/pr

In [2]:
# Load MNIST dataset
MNIST_train = pd.read_csv('~/data/mnist_train.csv', delimiter=",",header=None).values
MNIST_test  = pd.read_csv('~/data/mnist_test.csv', delimiter=",",header=None).values

# Prepare data (normalized onto [0,1])
Y_train = MNIST_train[0:10000,0]
X_train = MNIST_train[0:10000,1:]/255.0

Y_test = MNIST_test[:,0]
X_test = MNIST_test[:,1:]/255.0

In [ ]:
# Visualize individual pics
i=10
print(Y_train[i])
I=np.reshape(X_train[i], (28,28))
plt.imshow(I, interpolation='nearest',  cmap='gray_r')
plt.show()

# Logistic regression baseline

In [ ]:
# Logistic regression baseline
logreg = linear_model.LogisticRegression(multi_class='multinomial',solver='lbfgs')

logreg.fit(X_train,Y_train)


In [ ]:
# On train set

P=logreg.predict(X_train)

print(classification_report(Y_train,P))
print(confusion_matrix(Y_train, P))


In [ ]:
# On test set
P=logreg.predict(X_test)

print(classification_report(Y_test,P))
print(confusion_matrix(Y_test, P))


# Linear regression base line

In [7]:
linreg = linear_model.LinearRegression()

linreg.fit(X_train,Y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [10]:
# On train set

P=np.round(linreg.predict(X_train))

print(classification_report(Y_train,P))
print(confusion_matrix(Y_train, P))

             precision    recall  f1-score   support

       -2.0       0.00      0.00      0.00         0
       -1.0       0.00      0.00      0.00         0
        0.0       0.81      0.23      0.36      1001
        1.0       0.39      0.26      0.31      1127
        2.0       0.21      0.27      0.23       991
        3.0       0.28      0.34      0.31      1032
        4.0       0.15      0.21      0.17       980
        5.0       0.17      0.31      0.22       863
        6.0       0.22      0.29      0.25      1014
        7.0       0.30      0.30      0.30      1070
        8.0       0.15      0.11      0.13       944
        9.0       0.57      0.15      0.24       978
       10.0       0.00      0.00      0.00         0
       11.0       0.00      0.00      0.00         0
       12.0       0.00      0.00      0.00         0

avg / total       0.33      0.25      0.26     10000

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0

In [11]:
# On test set
P=linreg.predict(X_test)

print(classification_report(Y_test,P))
print(confusion_matrix(Y_test, P))


ValueError: Mix type of y not allowed, got types set(['continuous', 'multiclass'])

# Linear regression via CMA

In [ ]:
M = mdl.Model()
M.add(layers.Linear(784,1,paramBound=2))

minim = minimizer.CMA(False)
sol=minim.train(mse(), M, np.transpose(X_train), np.transpose(Y_train), maxiter=1000)

In [ ]:
# On train set
P=np.abs(np.round(np.real(M.predict(np.transpose(X_train)))))


print(classification_report(Y_train,P.T))
print(confusion_matrix(Y_train, P.T))

In [ ]:
# On test set
P=np.abs(np.round(np.real(M.predict(np.transpose(X_test)))))

print(classification_report(Y_test,P.T))
print(confusion_matrix(Y_test, P.T))

# Linear regression via SGD

In [3]:
M = mdl.Model()
M.add(layers.Linear(784,1))

minim = minimizer.SGD()
sol=minim.train(mse(), M, np.transpose(X_train), Y_train.reshape(1,len(Y_train)), lr=1, maxiter=10000, batch_size=1000)

Iteration 0 in 0.06(s), cost = 22.169172
Iteration 100 in 4.16(s), cost = 4.847189
Iteration 200 in 8.25(s), cost = 3.841688
Iteration 300 in 12.35(s), cost = 3.356877
Iteration 400 in 16.44(s), cost = 3.043384
Iteration 500 in 20.53(s), cost = 2.819812
Iteration 600 in 24.63(s), cost = 2.652400
Iteration 700 in 28.76(s), cost = 2.522701
Iteration 800 in 32.92(s), cost = 2.419384
Iteration 900 in 37.08(s), cost = 2.335093
Iteration 1000 in 41.23(s), cost = 2.264886
Iteration 1100 in 45.39(s), cost = 2.205358
Iteration 1200 in 49.54(s), cost = 2.154106
Iteration 1300 in 53.69(s), cost = 2.109398
Iteration 1400 in 57.88(s), cost = 2.069961
Iteration 1500 in 62.11(s), cost = 2.034841
Iteration 1600 in 66.27(s), cost = 2.003310
Iteration 1700 in 70.99(s), cost = 1.974802
Iteration 1800 in 76.27(s), cost = 1.948870
Iteration 1900 in 81.56(s), cost = 1.925156
Iteration 2000 in 86.91(s), cost = 1.903370
Iteration 2100 in 92.33(s), cost = 1.883271
Iteration 2200 in 97.81(s), cost = 1.864660
It

In [4]:
Y_train.shape

(10000,)

In [7]:
# On train set
P=np.abs(np.round(np.real(M.predict(np.transpose(X_train)))))


print(classification_report(Y_train,P.T))
print(confusion_matrix(Y_train, P.T))

             precision    recall  f1-score   support

        0.0       0.78      0.21      0.33      1001
        1.0       0.34      0.26      0.29      1127
        2.0       0.20      0.25      0.22       991
        3.0       0.26      0.32      0.29      1032
        4.0       0.13      0.19      0.16       980
        5.0       0.17      0.29      0.21       863
        6.0       0.19      0.26      0.22      1014
        7.0       0.29      0.30      0.30      1070
        8.0       0.13      0.09      0.11       944
        9.0       0.50      0.13      0.21       978
       10.0       0.00      0.00      0.00         0
       11.0       0.00      0.00      0.00         0
       12.0       0.00      0.00      0.00         0

avg / total       0.30      0.23      0.24     10000

[[207 368 238 124  34  16  10   4   0   0   0   0   0]
 [  6 288 520 192  85  23  12   1   0   0   0   0   0]
 [ 35 124 252 283 193  76  21   3   4   0   0   0   0]
 [ 12  55 196 329 261 103  41  18   9

In [8]:
# On test set
P=np.abs(np.round(np.real(M.predict(np.transpose(X_test)))))

print(classification_report(Y_test,P.T))
print(confusion_matrix(Y_test, P.T))


             precision    recall  f1-score   support

        0.0       0.72      0.18      0.29       980
        1.0       0.32      0.24      0.27      1135
        2.0       0.16      0.20      0.18      1032
        3.0       0.23      0.27      0.25      1010
        4.0       0.14      0.19      0.16       982
        5.0       0.17      0.29      0.22       892
        6.0       0.17      0.25      0.20       958
        7.0       0.24      0.26      0.25      1028
        8.0       0.17      0.12      0.14       974
        9.0       0.47      0.10      0.16      1009
       10.0       0.00      0.00      0.00         0
       11.0       0.00      0.00      0.00         0
       12.0       0.00      0.00      0.00         0

avg / total       0.28      0.21      0.21     10000

[[181 335 254 131  47  22   7   3   0   0   0   0   0]
 [  2 273 543 207  70  31   7   2   0   0   0   0   0]
 [ 53 157 205 270 201 108  24  10   3   1   0   0   0]
 [ 14  67 198 276 259 130  42  12  10

# E(h|v) via SGD

In [16]:
M = mdl.Model()
M.add(layers.DiagExpectationUnitLayer(784,1))
#M.add(layers.DiagExpectationUnitLayer(10,1))

minim = minimizer.SGD()
sol=minim.train(mse(), M, np.transpose(X_train), Y_train.reshape(1,Y_train.shape[0]),lr=100,momentum=0.1, nesterov=True, maxiter=1000, batch_size=1000)

Iteration 0 in 0.49(s), cost = 4.510531
Iteration 100 in 49.25(s), cost = 1.679647
Iteration 200 in 98.69(s), cost = 1.533806
Iteration 300 in 148.29(s), cost = 1.480881
Iteration 400 in 197.82(s), cost = 1.454672
Iteration 500 in 247.33(s), cost = 1.439183
Iteration 600 in 296.74(s), cost = 1.429124
Iteration 700 in 346.09(s), cost = 1.422152
Iteration 800 in 395.40(s), cost = 1.417085
Iteration 900 in 438.21(s), cost = 1.413265
('Cost: ', (1.4103265375527827+0j))
('Sol: ', array([-7.50912005e+00+0.j, -5.40483139e-01+0.j, -4.23662437e-01+0.j,
       -8.50774267e-01+0.j,  2.94870616e-01+0.j,  7.46804247e-01+0.j,
       -5.01419768e-01+0.j, -1.36482997e-01+0.j,  6.83261794e-02+0.j,
        5.62876742e-01+0.j,  8.48431384e-01+0.j,  9.54515643e-01+0.j,
        9.68861580e-01+0.j, -4.53871189e-01+0.j,  3.37372701e-01+0.j,
        9.10277193e-01+0.j,  5.03094414e-01+0.j,  6.55547704e-01+0.j,
       -8.54890079e-01+0.j,  5.57729368e-01+0.j,  5.58282626e-01+0.j,
        2.24974221e-01+0.j, -8

In [19]:
# On train set
P=np.abs(np.round(np.real(M.predict(np.transpose(X_train)))))


print(classification_report(Y_train,P.T))
print(confusion_matrix(Y_train, P.T))

             precision    recall  f1-score   support

        0.0       0.82      0.21      0.33      1001
        1.0       0.32      0.23      0.27      1127
        2.0       0.20      0.26      0.22       991
        3.0       0.27      0.32      0.30      1032
        4.0       0.13      0.17      0.15       980
        5.0       0.18      0.31      0.23       863
        6.0       0.21      0.28      0.24      1014
        7.0       0.29      0.31      0.30      1070
        8.0       0.14      0.11      0.12       944
        9.0       0.55      0.16      0.25       978
       10.0       0.00      0.00      0.00         0
       11.0       0.00      0.00      0.00         0
       12.0       0.00      0.00      0.00         0

avg / total       0.31      0.24      0.24     10000

[[207 382 247  98  36  19  11   1   0   0   0   0   0]
 [  3 260 546 201  81  30   4   1   1   0   0   0   0]
 [ 29 108 256 304 189  80  16   8   1   0   0   0   0]
 [  9  50 181 333 282 108  34  21   7

In [20]:
# On test set
P=np.abs(np.round(np.real(M.predict(np.transpose(X_test)))))

print(classification_report(Y_test,P.T))
print(confusion_matrix(Y_test, P.T))



             precision    recall  f1-score   support

        0.0       0.78      0.19      0.30       980
        1.0       0.32      0.24      0.28      1135
        2.0       0.18      0.22      0.20      1032
        3.0       0.25      0.29      0.27      1010
        4.0       0.14      0.19      0.16       982
        5.0       0.18      0.30      0.22       892
        6.0       0.17      0.24      0.20       958
        7.0       0.24      0.27      0.25      1028
        8.0       0.15      0.12      0.13       974
        9.0       0.50      0.12      0.19      1009
       10.0       0.00      0.00      0.00         0
       11.0       0.00      0.00      0.00         0
       12.0       0.00      0.00      0.00         0
       13.0       0.00      0.00      0.00         0

avg / total       0.29      0.22      0.22     10000

[[184 372 226 120  45  23   7   3   0   0   0   0   0   0]
 [  2 271 545 201  74  35   6   1   0   0   0   0   0   0]
 [ 35 130 224 293 215 103  21  

# E(h|v) via CMA

In [ ]:
M = mdl.Model()
M.add(layers.DiagExpectationUnitLayer(784,20, phase=1j))
M.add(layers.DiagExpectationUnitLayer(20,1, phase=1j))

minim = minimizer.CMA(False)
sol=minim.train(mse(), M, np.transpose(X_train), np.transpose(Y_train), maxiter=100)

In [ ]:
# On train set
P=np.abs(np.round(np.real(M.predict(np.transpose(X_train)))))


print(classification_report(Y_train,P.T))
print(confusion_matrix(Y_train, P.T))

In [ ]:
# On test set
P=np.abs(np.round(np.real(M.predict(np.transpose(X_test)))))

print(classification_report(Y_test,P.T))
print(confusion_matrix(Y_test, P.T))
